In [ ]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [ ]:
start_case = 500
case_num = 500

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]


def objective(trial: optuna.trial.Trial) -> float:
    min_k = trial.suggest_float("max_k", low=0.5, high=3.0)
    max_k = trial.suggest_float("min_k", low=min_k + 0.5, high=min_k+6.0)
    k_p = trial.suggest_float("k_p", low=0.2, high=3.0)

    args = " ".join(map(str, [min_k, max_k, k_p]))

    solver_path = "./target/release/ahc030"
    solver_cmd = f"./tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-{args}"
    print(solver_version)
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evaluate_overall_relative_score()
    return score_df.relative_score.median()


subprocess.run("cargo build --features local --release --manifest-path=../Cargo.toml", shell=True)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc030.db",
    study_name="tune-query-size-02",
    load_if_exists=True,
)
print("start optimize")
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)